## Introduction

The purpose of this notebook is to demonstrate how to efficiently tune hyperparameters using simulated annealing (the functions available in simulated_annealing.py in-specific). We will use the dataset containing credit card transactions of european card holders in the month of Sep 2013 with the aim of flagging fraudulent transactions. The dataset can be accessed [here](https://www.kaggle.com/mlg-ulb/creditcardfraud). An XGBoost classifier will be used for this exercise as the number of hyperparameters are high and an extensive grid search can be computationally expensive.

## Import Libraries

In [14]:
from collections import OrderedDict
from random import random

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

import matplotlib.pyplot as plt
%matplotlib inline

## Helper Functions

In [2]:
def display_all(df):
    with pd.option_context('display.max_rows', 1000):
        with pd.option_context('display.max_columns', 1000):
            display(df)

## Read Data

In [3]:
raw_df = pd.read_csv('data/creditcardfraud.zip', compression='zip')

In [4]:
raw_df.shape

(284807, 31)

In [5]:
display_all(raw_df.head())

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
display_all(raw_df.tail())

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,-1.040458,-0.031513,-0.188093,-0.084316,0.041333,-0.302620,-0.660377,0.167430,-0.256117,0.382948,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [7]:
raw_df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

A quick glance at the dataset shows the presence of following variables:
- Time - Seconds from the first transaction
- V1:V28 - Principal components of the transaction features (Original data could not be shared due to confidentiality issues)
- Amount - Transaction amount
- Class - Flag to indicate wheather a transaction is fraudulent or not (1 implies fraud)

As the primary goal of this exercise is to show the application of simulated annealing, we are not going to look at the data closely. In addition, since we have ample data, instead of cross validation a single validation set will be used to tune the hyper-parameters and the final model performance can be evaluated on a hold-out test set.

## Split Dataset

We will split the data into train-test-validation sets with a 60:20:20 ratio.

In [8]:
x_tr, test = train_test_split(raw_df, test_size=0.2, shuffle=True)

In [9]:
train, valid = train_test_split(x_tr, test_size=0.25, shuffle=True)

In [10]:
xtrain, ytrain = train.drop('Class', axis=1), train['Class']
xvalid, yvalid = valid.drop('Class', axis=1), valid['Class']
xtest, ytest = test.drop('Class', axis=1), test['Class']

## Define Parameter Space

The simulated annealing functions defined take in two dictionaries of parameters:

- Static parameters that are kept unchanged through out the tuning process - For the 
- Parameters to be tuned

Parameter search space is chosen based on the following articles:
- [XGBoost Notes on Tuning](http://xgboost.readthedocs.io/en/latest/how_to/param_tuning.html)
- [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

Brief overview of the parameters:

- eval_metric - Metric to be used to measure model performance
- min_child_weight - The minimum sum of weights of all observations required in a child
- seed - random number seed to generate reproducible results
- max_depth - Maximum tree depth the individual learners can grow upto
- subsample - Fraction of observations used to train individual learners
- colsample_bytree - Fraction of columns considered for each split
- learning_rate - shrinkage weights of the weights
- gamma - Minimum loss reduction required to make a split
- scale_pos_weight - controls balance of positive and negative ratio
- silent - Parameter to that controls whether the model prints messages while running
- objective - Objective function used for learning
- n_estimators - Number of trees
- n_jobs - Number of parallel threds to run

In [11]:
# Parameters that are kept constant during the tuning process
param = {
    'silent': False,
    'min_child_weight': 1,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'seed': 42,
    'n_estimators': 20,
    'n_jobs': -1
}

In [15]:
# Parameter search space
tune_dic = OrderedDict()
tune_dic['max_depth'] = [5, 10, 15, 20, 25]
tune_dic['subsample'] = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
tune_dic['colsample_bytree'] = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
tune_dic['learning_rate'] = [0.01, 0.05, 0.10, 0.20, 0.30, 0.40]
tune_dic['gamma'] = [0.00, 0.05, 0.10, 0.15, 0.20]
tune_dic['scale_pos_weight'] = [30, 40, 50, 300, 400, 500, 600, 700]

Next up, let us define a function that takes in parameter dictionaries, train/validation datasets and an evaluation metric and returns the model and metric computed on the validation set. This function is iteratively called in the annealing process.

In [16]:
# Function to train model
def train_model(curr_params, param, Xtrain, Xvalid, Ytrain, Yvalid, metric=f1_score):
    """
    Train the model with given set of hyperparameters
    curr_params - Dict of hyperparameters and chosen values
    param - Dict of hyperparameters that are kept constant
    Xtrain - Train Data
    Xvalid - Validation Data
    Ytrain - Train labels
    Yvalid - Validaion labels
    metric - Metric to compute model performance on
    """
    params_copy = param.copy()
    params_copy.update(curr_params)
    model = XGBClassifier(**params_copy)
    model.fit(Xtrain, Ytrain)
    preds = model.predict(Xvalid)
    metric_val = metric(Yvalid, preds)
    
    return model, metric_val

Define a function to choose next set of parameters from the vicinity of current parameters. Randomly select one parameter to update and choose either previous or next value from the search space for that parameter. If the current parameter happens to be first or last value in the list, second or second to last values will be chosen respectively.

In [17]:
def choose_params(tune_dic, curr_params=None):
    """
    Function to choose parameters for next iteration
    Inputs:
    tune_dic - Dict of Hyperparameter search space
    curr_params - Dict of current hyperparameters
    Output:
    Dictionary of parameters
    """
    if curr_params:
        next_params = curr_params.copy()
        param_to_update = np.random.choice(list(tune_dic.keys()))
        param_vals = tune_dic[param_to_update]
        curr_index = param_vals.index(curr_params[param_to_update])
        if curr_index == 0:
            next_params[param_to_update] = param_vals[1]
        elif curr_index == len(param_vals) - 1:
            next_params[param_to_update] = param_vals[curr_index - 1]
        else:
            next_params[param_to_update] = \
                param_vals[curr_index + np.random.choice([-1, 1])]
    else:
        next_params = dict()
        for k, v in tune_dic.items():
            next_params[k] = np.random.choice(v)

    return next_params

Define function to perform simulated annealing. Takes in parameter dictionaries, datasets, training function and annealing parameters. Steps involved in simulated annealing process are:

1. Initialize/Update parameters
2. Repeat step 1 if the updated parameters have already been used (use a hash function)
3. Fit a model and compute metric
4. If the metric value is an improvement over previous value, accept the parameters and go to step 1
5. If the metric value is not an improvement from previous value, accept the parameters with probability defined by annealing function ($e^{-\beta\Delta f/ T}$). In case the parameters are rejected, use parameters from previous iteration to create parameters for next iteration.

Annealing Function $e^{-\beta\Delta f/ T}$
- beta - constant term to normalize the values inside the exponential function
- T - Temperature, reduced after at a rate $\alpha$ for each fixed number of iterations
- $\Delta f$ - previous metric value - current metric value

Parameters $\alpha$, $\beta$ and T are chosen such that the probability of accepting a decrease in score is high initially but decreases with iterations. This will allow for a wider search space for the first few iterations and restrict the updates in later iterations.

In [18]:
def simulate_annealing(tune_dic,
                       const_param,
                       X_train,
                       X_valid,
                       Y_train,
                       Y_valid,
                       fn_train,
                       maxiters=100,
                       alpha=0.85,
                       beta=1.3,
                       T=0.40,
                       update_iters=5):
    """
    Function to perform hyperparameter search using simulated annealing
    Inputs:
    tune_dic - Dictionary of Hyperparameter search space
    const_param - Static parameters of the model
    Xtrain - Train Data
    Xvalid - Validation Data
    Ytrain - Train labels
    Yvalid - Validaion labels
    fn_train - Function to train the model
        (Should return model and metric value as tuple, sample commented above)
    maxiters - Number of iterations to perform the parameter search
    alpha - factor to reduce temperature
    beta - constant in probability estimate
    T - Initial temperature
    update_iters - # of iterations required to update temperature
    Output:
    Dataframe of the parameters explored and corresponding model performance
    """
    columns = [*tune_dic.keys()] + ['Metric', 'Best Metric']
    results = pd.DataFrame(index=range(maxiters), columns=columns)
    best_metric = -1.
    prev_metric = -1.
    prev_params = None
    best_params = dict()
    weights = list(map(lambda x: 10**x, list(range(len(tune_dic)))))
    hash_values = set()

    for i in range(maxiters):
        print('Starting Iteration {}'.format(i))
        while True:
            curr_params = choose_params(tune_dic, prev_params)
            indices = [tune_dic[k].index(v) for k, v in curr_params.items()]
            hash_val = sum([i * j for (i, j) in zip(weights, indices)])
            if hash_val in hash_values:
                print('Combination revisited')
            else:
                hash_values.add(hash_val)
                break

        model, metric = fn_train(curr_params, const_param, X_train,
                                 X_valid, Y_train, Y_valid)

        if metric > prev_metric:
            print('Local Improvement in metric from {:8.4f} to {:8.4f} '
                  .format(prev_metric, metric) + ' - parameters accepted')
            prev_params = curr_params.copy()
            prev_metric = metric

            if metric > best_metric:
                print('Global improvement in metric from {:8.4f} to {:8.4f} '
                      .format(best_metric, metric) +
                      ' - best parameters updated')
                best_metric = metric
                best_params = curr_params.copy()
                best_model = model
        else:
            rnd = np.random.uniform()
            diff = metric - prev_metric
            threshold = np.exp(beta * diff / T)
            if rnd < threshold:
                print('No Improvement but parameters accepted. Metric change' +
                      ': {:8.4f} threshold: {:6.4f} random number: {:6.4f}'
                      .format(diff, threshold, rnd))
                prev_metric = metric
                prev_params = curr_params
            else:
                print('No Improvement and parameters rejected. Metric change' +
                      ': {:8.4f} threshold: {:6.4f} random number: {:6.4f}'
                      .format(diff, threshold, rnd))

        results.loc[i, list(curr_params.keys())] = list(curr_params.values())
        results.loc[i, 'Metric'] = metric
        results.loc[i, 'Best Metric'] = best_metric

        if i % update_iters == 0:
            T = alpha * T

    return results, best_model

In [ ]:
res, best_model = simulate_annealing(tune_dic, param, xtrain,
                                     xvalid, ytrain, yvalid,
                                     train_model, maxiters=100,
                                     beta=15)

Starting Iteration 0


/Users/santhoshhari/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Local Improvement in metric from  -1.0000 to   0.7280  - parameters accepted
Global improvement in metric from  -1.0000 to   0.7280  - best parameters updated
Starting Iteration 1


/Users/santhoshhari/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Local Improvement in metric from   0.7280 to   0.7368  - parameters accepted
Global improvement in metric from   0.7280 to   0.7368  - best parameters updated
Starting Iteration 2


...

Local Improvement in metric from   0.8263 to   0.8372  - parameters accepted
Starting Iteration 20


/Users/santhoshhari/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


No Improvement but parameters accepted. Metric change:   0.0000 threshold: 1.0000 random number: 0.2314
Starting Iteration 21
Combination revisited


/Users/santhoshhari/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


No Improvement and parameters rejected. Metric change:  -0.0015 threshold: 0.8788 random number: 0.9250
Starting Iteration 22


In [22]:
res[res.Metric==res['Best Metric'].max()]

,max_depth,subsample,colsample_bytree,learning_rate,gamma,scale_pos_weight,Metric,Best Metric
58,15,0.9,0.8,0.2,0.15,30,0.875,0.875
61,15,0.9,0.8,0.2,0.2,30,0.875,0.875
62,10,0.9,0.8,0.2,0.2,30,0.875,0.875
65,10,0.9,0.8,0.2,0.15,30,0.875,0.875
67,10,0.9,0.8,0.2,0.1,30,0.875,0.875
69,15,0.9,0.8,0.2,0.1,30,0.875,0.875
72,15,0.9,0.8,0.2,0.05,30,0.875,0.875
74,20,0.9,0.8,0.2,0.05,30,0.875,0.875
76,25,0.9,0.8,0.2,0.05,30,0.875,0.875
77,25,0.9,0.8,0.2,0,30,0.875,0.875


In [ ]:
preds = best_model.predict(xtest)
f1_score(ytest, preds)

## End Notes

- If number of iterations is not significantly lower than the search space, current implementation will result in too many repitations and delays.
- Once a set of optimal parameters are identified 1-2 rounds of annealing can be performed by refining the search space.